In [2]:
"""
univ3-volume.ipynb

Goal with this notebook is to plot the volume
of a given pool over time.
"""
# switch to univ3-ape wd first for ape project to work properly
import os
from pathlib import Path

if Path(*Path(os.getcwd()).parts[-3:]) == Path('univ3-ape/notebook/queries'):
    os.chdir('../..')

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import typing as tp
from ape import accounts, chain, Contract, networks, project

In [5]:
# SEE: https://gist.github.com/banteg/dcf6082ff7fc6ad51ce220146f29d9ff
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

In [6]:
univ3 = project.dependencies['UniswapV3Core']['main']

In [7]:
# SEE: https://docs.uniswap.org/contracts/v3/reference/deployments
factory = Contract("0x1F98431c8aD98523631AE4a59f267346ea31F984")
factory

<UniswapV3Factory 0x1F98431c8aD98523631AE4a59f267346ea31F984>

In [8]:
head_block = chain.blocks.head.number

In [14]:
deploy_block = 12369621  # block when factory deployed
step = 300  # volume binned hourly
start_block = head_block - 7200 * 7 # last 7 days ... TODO: last 6 months

In [15]:
pool = univ3.UniswapV3Pool.at("0xa3f558aebAecAf0e11cA4b2199cC5Ed341edfd74")  # LDO/ETH 0.3%

In [17]:
print(f"token0: {pool.token0()}, token1: {pool.token1()}, fee: {pool.fee()}")

token0: 0x5A98FcBEA516Cf06857215779Fd812CA3beF1B32, token1: 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2, fee: 3000


In [18]:
slot0 = pool.slot0()
slot0

slot0_return(sqrtPriceX96=3263475611440845534671539341, tick=-63794, observationIndex=0, observationCardinality=1, observationCardinalityNext=1, feeProtocol=0, unlocked=True)

In [25]:
# utility functions to unfold event arguments into separate columns
def unfold_event_args(key: str, event_args: dict) -> tp.Any:
    return event_args[key]

def unfold(df: pd.DataFrame):
    for key in df.iloc[-1]['event_arguments'].keys():
        df[key] = df['event_arguments'].apply(lambda ev: unfold_event_args(key, ev))

In [23]:
# query Swap event data
%time df_swaps = pool.Swap.query('*', start_block=start_block)

INFO: Cache database has not been initialized
CPU times: user 4.7 s, sys: 560 ms, total: 5.26 s
Wall time: 13.5 s


In [26]:
unfold(df_swaps)

In [29]:
# per swap volume amounts
df_swaps['vol_amount0'] = abs(df_swaps['amount0'])
df_swaps['vol_amount1'] = abs(df_swaps['amount1'])

In [30]:
df_swaps

,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index,sender,recipient,amount0,amount1,sqrtPriceX96,liquidity,tick,vol_amount0,vol_amount1
0,Swap,0xa3f558aebAecAf0e11cA4b2199cC5Ed341edfd74,{'sender': '0xf8b721bFf6Bf7095a0E10791cE8f998b...,0xb88b5a322a41c8e3511591594e92a2f72c378bab6335...,16587846,0xd403fc317ef3acc36cf58bec7819cfd68d8b5d0334d9...,2,0,0xf8b721bFf6Bf7095a0E10791cE8f998baa254Fd0,0xf8b721bFf6Bf7095a0E10791cE8f998baa254Fd0,35736283141913262704780,-60514123646537288331,3261127549363715359277171725,594881493042756452768598,-63809,35736283141913262704780,60514123646537288331
1,Swap,0xa3f558aebAecAf0e11cA4b2199cC5Ed341edfd74,{'sender': '0xf8b721bFf6Bf7095a0E10791cE8f998b...,0x4fe78078b8b222813892afb852eae73f37ce332b88fd...,16587847,0x64c9d879fc5b55052480f46e96f0ba9e357d8c6d0d88...,2,0,0xf8b721bFf6Bf7095a0E10791cE8f998baa254Fd0,0xf8b721bFf6Bf7095a0E10791cE8f998baa254Fd0,12791363809106260207923,-21587649410604961115,3258252439252193629533383776,594881493042756452768598,-63827,12791363809106260207923,21587649410604961115
2,Swap,0xa3f558aebAecAf0e11cA4b2199cC5Ed341edfd74,{'sender': '0xA69babEF1cA67A37Ffaf7a485DfFF338...,0xee38e54ed2c888d904c19b4f0e365a51f2d94552c1b1...,16587847,0x64c9d879fc5b55052480f46e96f0ba9e357d8c6d0d88...,30,7,0xA69babEF1cA67A37Ffaf7a485DfFF3382056e78C,0x56178a0d5F301bAf6CF3e1Cd53d9863437345Bf9,18372017546129254242577,-30939473215931513435,3254135198746335788121047339,595973298096764907599134,-63852,18372017546129254242577,30939473215931513435
3,Swap,0xa3f558aebAecAf0e11cA4b2199cC5Ed341edfd74,{'sender': '0xA69babEF1cA67A37Ffaf7a485DfFF338...,0xc0a808b7cda205e4e68a2f0b5494cccb930d6cda9d66...,16587848,0x2c35b1ad329f5bb3219923522853180590b5b2a2f1b9...,2,0,0xA69babEF1cA67A37Ffaf7a485DfFF3382056e78C,0x56178a0d5F301bAf6CF3e1Cd53d9863437345Bf9,8363327161186713921341,-14058428169679966578,3252266283754205266958007025,595973298096764907599134,-63863,8363327161186713921341,14058428169679966578
4,Swap,0xa3f558aebAecAf0e11cA4b2199cC5Ed341edfd74,{'sender': '0xEf1c6E67703c7BD7107eed8303Fbe6EC...,0x436704e4a6e3da4a2b1d667be9f4ee02535929616ecd...,16587850,0xb63b79d9f1cfe2538370df546bb2ab19f887e2f4d3f0...,176,65,0xEf1c6E67703c7BD7107eed8303Fbe6EC2554BF6B,0x5fdd94014e63664eD3053C869515F5a82544Db2b,-2957764901169252608535,5000000000000000000,3252928985239984721249710138,595973298096764907599134,-63859,2957764901169252608535,5000000000000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6194,Swap,0xa3f558aebAecAf0e11cA4b2199cC5Ed341edfd74,{'sender': '0xEf1c6E67703c7BD7107eed8303Fbe6EC...,0xb3bbb9763a3708b335d8966182b287fdff1b21f7dbfe...,16638158,0x918dd85c90def6ca07827e426366d5df4dc84a395ab1...,8,3,0xEf1c6E67703c7BD7107eed8303Fbe6EC2554BF6B,0xEf1c6E67703c7BD7107eed8303Fbe6EC2554BF6B,10000000000000000000000,-16875877443103653731,3258786984099334879267212553,815569498020751480434149,-63823,10000000000000000000000,16875877443103653731
6195,Swap,0xa3f558aebAecAf0e11cA4b2199cC5Ed341edfd74,{'sender': '0xA69babEF1cA67A37Ffaf7a485DfFF338...,0x955dc193cff64312140bda7964367622c0775c6564d9...,16638216,0x55d47ee7300842aebfd9d34adcc819d50cd55243ce49...,189,20,0xA69babEF1cA67A37Ffaf7a485DfFF3382056e78C,0x56178a0d5F301bAf6CF3e1Cd53d9863437345Bf9,5255835799055271150451,-8862882028200853848,3257926003080084962024849105,815569498020751480434149,-63829,5255835799055271150451,8862882028200853848
6196,Swap,0xa3f558aebAecAf0e11cA4b2199cC5Ed341edfd74,{'sender': '0x000000000dFDe7deaF24138722987c9a...,0x5bbdf8f431f024ac113528b45ad205a91caedaf3ef7d...,16638228,0xe37995d8aaafafdfecfbd380e81b3b99f8f4f6396fe3...,59,27,0x000000000dFDe7deaF24138722987c9a6991e2D4,0x000000000dFDe7deaF24138722987c9a6991e2D4,7004610090386025000000,-11804551608982803921,3256779254782663086846699139,815569498020751480434149,-63836,7004610090386025000000,11804551608982803921
6197,Swap,0xa3f558aebAecAf0e11cA4b2199cC5Ed341edfd74,{'sender': '0xf8b721bFf6Bf7095a0E10791cE8f998b...,0x7141e3253b

In [36]:
# filter only columns we care about
df_swaps_filtered = df_swaps.filter(items=['block_number', 'vol_amount0', 'vol_amount1'])
df_swaps_filtered

,block_number,vol_amount0,vol_amount1
0,16587846,35736283141913262704780,60514123646537288331
1,16587847,12791363809106260207923,21587649410604961115
2,16587847,18372017546129254242577,30939473215931513435
3,16587848,8363327161186713921341,14058428169679966578
4,16587850,2957764901169252608535,5000000000000000000
...,...,...,...
6194,16638158,10000000000000000000000,16875877443103653731
6195,16638216,5255835799055271150451,8862882028200853848
6196,16638228,7004610090386025000000,11804551608982803921
6197,16638250,6939816981652078905742,11765821945986658015


In [50]:
# sum all volume in a block
df_swaps_filtered_grouped = df_swaps_filtered.groupby('block_number').transform('sum')
df_swaps_filtered_grouped['block_number'] = df_swaps_filtered['block_number']
df_swaps_filtered_grouped = df_swaps_filtered_grouped.drop_duplicates('block_number')

In [51]:
# set block_number to index
df_swaps_filtered_grouped = df_swaps_filtered_grouped.set_index('block_number')
df_swaps_filtered_grouped

,vol_amount0,vol_amount1
block_number,,
16587846,35736283141913262704780,60514123646537288331
16587847,31163381355235514450500,52527122626536474550
16587848,8363327161186713921341,14058428169679966578
16587850,2957764901169252608535,5000000000000000000
16587851,1000000000000000000000,1680565047522800419
...,...,...
16638158,10000000000000000000000,16875877443103653731
16638216,5255835799055271150451,8862882028200853848
16638228,7004610090386025000000,11804551608982803921


In [62]:
# reindex by block number (1 block steps) and ffill price values 
new_index = range(df_swaps_filtered_grouped.index.min(), df_swaps_filtered_grouped.index.max()+1)
df_swaps_filtered_grouped_reindexed = df_swaps_filtered_grouped.reindex(new_index, fill_value=0)
df_swaps_filtered_grouped_reindexed

,vol_amount0,vol_amount1
block_number,,
16587846,35736283141913262704780,60514123646537288331
16587847,31163381355235514450500,52527122626536474550
16587848,8363327161186713921341,14058428169679966578
16587849,0,0
16587850,2957764901169252608535,5000000000000000000
...,...,...
16638247,0,0
16638248,0,0
16638249,0,0


In [70]:
s = (df_swaps_filtered_grouped_reindexed.index.to_series() / step).astype(int)
df_swaps_candled = df_swaps_filtered_grouped_reindexed.groupby(s).agg({'vol_amount0': 'sum', 'vol_amount1': 'sum'}).set_index(s.index[::step])
df_swaps_candled

,vol_amount0,vol_amount1
block_number,,
16587846,197816648523979484702314,332719383475942718095
16588146,594182467960871833443404,1030827557058667885570
16588446,2631275839255048029029181,4188080583254230273326
16588746,1820956110666604898074276,2952135413377088648014
16589046,917612867896854087062634,1569199276578865815851
...,...,...
16637046,91711723427174992141236,154747747950336691470
16637346,1528308467022700168906397,2558377482714510547176
16637646,1033951006099289345101503,1777154635016211629362
